In [1]:
import re
import sys
import traceback
from concurrent.futures import ThreadPoolExecutor, wait
from datetime import datetime
from queue import Queue

import pymongo
import tweepy
from kipp.decorator import debug_wrapper
from pymongo import MongoClient
from tweepy import API, OAuthHandler

executor = ThreadPoolExecutor(max_workers=10)


# sys.path.append(r'/Users/laisky/repo/laisky/ramjet/ramjet/settings')
sys.path.append(r"/opt/configs/ramjet")
sys.path
import prd

mongo = MongoClient(
    f"mongodb://{prd.MONGO_USER}:{prd.MONGO_PASSWD}@{prd.MONGO_HOST}:{prd.MONGO_PORT}/{prd.MONGO_DB}",
)
tweets = mongo["twitter"]["tweets"]
users = mongo["twitter"]["users"]

auth = OAuthHandler(prd.CONSUMER_KEY, prd.CONSUMER_SECRET)
auth.set_access_token(prd.ACCESS_TOKEN, prd.ACCESS_TOKEN_SECRET)
api = API(auth, wait_on_rate_limit=True, parser=tweepy.parsers.JSONParser())
# api.get_user()
api.get_lists()

[{'id': 1506792312554278912,
  'id_str': '1506792312554278912',
  'name': 'Ukrain',
  'uri': '/ChiangMaiGreat/lists/1506792312554278912',
  'subscriber_count': 0,
  'member_count': 51,
  'mode': 'public',
  'description': '',
  'slug': '1506792312554278912',
  'full_name': '@ChiangMaiGreat/1506792312554278912',
  'created_at': 'Thu Mar 24 00:37:36 +0000 2022',
  'following': True,
  'user': {'id': 1354782101308985349,
   'id_str': '1354782101308985349',
   'name': 'เชียงใหม่🇹🇭',
   'screen_name': 'ChiangMaiGreat',
   'location': '距离你 147.4 亿光年',
   'description': '清迈房产、移民、伴游、K12 国际教育咨询',
   'url': 'https://t.co/KJdQLlpZrS',
   'entities': {'url': {'urls': [{'url': 'https://t.co/KJdQLlpZrS',
       'expanded_url': 'http://talkingsallowed.gov.co.th',
       'display_url': 'talkingsallowed.gov.co.th',
       'indices': [0, 23]}]},
    'description': {'urls': []}},
   'protected': False,
   'followers_count': 6144,
   'friends_count': 220,
   'listed_count': 56,
   'created_at': 'Thu Jan 2

In [12]:
api.get_status('1627687320718839808', tweet_mode="extended")

# api.user_timeline?


# .search(q='from:ChiangMaiGreate', since_id='1627687320718839808', tweet_mode="extended")

{'created_at': 'Mon Feb 20 15:11:12 +0000 2023',
 'id': 1627687320718839808,
 'id_str': '1627687320718839808',
 'full_text': '历史学家们一直争论历史究竟是结构化客观演化，还是历史人物的主观影响。\n\n我觉得中国的三年抗疫灾害就是个很好的例子。从客观的历史进程来说人类必然走向群体免疫。但于与此同时，某些人的一意孤行亲自指挥让中国的这一客观进程比其他国家晚了一年。… https://t.co/Uo9JOyMrOj',
 'truncated': False,
 'display_text_range': [0, 142],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/Uo9JOyMrOj',
    'expanded_url': 'https://twitter.com/i/web/status/1627687320718839808',
    'display_url': 'twitter.com/i/web/status/1…',
    'indices': [119, 142]}]},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1354782101308985349,
  'id_str': '1354782101308985349',
  'name': 'เชียงใหม่🇹🇭',
  'screen_name': 'ChiangMaiGreat',
  'loc

In [ ]:
api.user_timeline?

In [3]:
api.user_timeline(tweet_mode='extended', max_id=1372203983008276482)

[]

In [ ]:
# index

# tweets.create_index([('created_at', pymongo.DESCENDING)])
# tweets.create_index([('id_str', pymongo.DESCENDING)])
# tweets.create_index([('created_at', pymongo.DESCENDING)])
tweets.create_index([('user.id_str', pymongo.DESCENDING)])
# users.create_index([('id', pymongo.DESCENDING)])

In [2]:
# tweets.update_many({}, {"$set": {"deleted": False}})

In [2]:
# delete tweets

# @debug_wrapper
def is_status_tobe_delete(tweet):
#     if len(tweet.get("entities", {}).get("hashtags", [])) >= 1:
#         return False

    if tweet["created_at"] > datetime(2022, 5, 1):
        return False

#     if tweet.get("in_reply_to_status_id") is not None:
#         return False

#     if tweets.count_documents({"in_reply_to_status_id": tweet["id"]}) > 0:
#         return False

    return True


@debug_wrapper
def run_delete(user_id_str: str):
    delete_q = Queue(maxsize=50)
    fs = []
    fs.append(executor.submit(gen_tweet, delete_q, user_id_str))
    for _ in range(10):
        fs.append(executor.submit(delete_tweet, delete_q))

    wait(fs)


@debug_wrapper
def gen_tweet(q: Queue, user_id_str):
    for tweet in tweets.find(
        {
            "deleted": {"$ne": True}, 
            "user.id_str": user_id_str,
        }).sort("created_at", 1):
#         print(tweet)
        
        q.put(tweet)


@debug_wrapper
def delete_tweet(q: Queue):
    while 1:
        tweet = q.get()
        if not is_status_tobe_delete(tweet):
            print(f"pass tweet {tweet['id']} {tweet['created_at']}")
            continue

        try:
            api.destroy_status(tweet["id"])
        except tweepy.errors.TweepyException as err:
            if "No status found with that ID." in f"{err}":
                # deleted
                tweets.update_one(
                    {"_id": tweet["_id"]},
                    {"$set": {"deleted": True}},
                )
                print(f"mark {tweet['id']} {tweet['created_at']} deleted")
                continue

            traceback.print_exc()
        except Exception:
            traceback.print_exc()
        else:

            tweets.update_one(
                {"_id": tweet["_id"]},
                {"$set": {"deleted": True}},
            )

            #         n_del += 1
            print("delete", tweet["id"], tweet["created_at"])


#         if n % 1000 == 0:
#             print(f">> scan {n} statuses")
#         if n_del % 100 == 0:
#             print(f">> delete {n_del} statuses")


run_delete("1354782101308985349")


mark 1354806798608338951 2021-01-28 15:01:26 deleted
delete 1354786874829672451 2021-01-28 13:42:16
delete 1354786354807234585 2021-01-28 13:40:12
delete 1354786639080394755 2021-01-28 13:41:20
delete 1354785984269848577 2021-01-28 13:38:44
delete 1354959704250740746 2021-01-29 01:09:02
delete 1354787086310641666 2021-01-28 13:43:06
delete 1354785373885329418 2021-01-28 13:36:18
delete 1354959568162316291 2021-01-29 01:08:29
delete 1354959799885041672 2021-01-29 01:09:24
delete 1354960197899280391 2021-01-29 01:10:59
delete 1354960524404871189 2021-01-29 01:12:17
delete 1354963492210532363 2021-01-29 01:24:05
delete 1355016550235836420 2021-01-29 04:54:55
delete 1354961456270217216 2021-01-29 01:15:59
delete 1354963859317026821 2021-01-29 01:25:32
delete 1354960699366105102 2021-01-29 01:12:59
delete 1354960476862509060 2021-01-29 01:12:06
delete 1355016953694355456 2021-01-29 04:56:31
delete 1355017279323299842 2021-01-29 04:57:49
delete 1355017422072209408 2021-01-29 04:58:23
delete 

Traceback (most recent call last):
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/urllib3/connectionpool.py", line 696, in urlopen
    self._prepare_proxy(conn)
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/urllib3/connectionpool.py", line 964, in _prepare_proxy
    conn.connect()
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/urllib3/connection.py", line 371, in connect
    self._tunnel()
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/http/client.py", line 929, in _tunnel
    raise OSError(f"Tunnel connection failed: {code} {message.strip()}")
OSError: Tunnel connection failed: 502 ERROR

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/requests/adapters.py", line 439, in send
    resp = conn.urlopen(
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/urllib3/

Traceback (most recent call last):
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/urllib3/connectionpool.py", line 696, in urlopen
    self._prepare_proxy(conn)
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/urllib3/connectionpool.py", line 964, in _prepare_proxy
    conn.connect()
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/urllib3/connection.py", line 371, in connect
    self._tunnel()
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/http/client.py", line 929, in _tunnel
    raise OSError(f"Tunnel connection failed: {code} {message.strip()}")
OSError: Tunnel connection failed: 502 ERROR

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/requests/adapters.py", line 439, in send
    resp = conn.urlopen(
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/urllib3/

Traceback (most recent call last):
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/urllib3/connectionpool.py", line 696, in urlopen
    self._prepare_proxy(conn)
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/urllib3/connectionpool.py", line 964, in _prepare_proxy
    conn.connect()
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/urllib3/connection.py", line 371, in connect
    self._tunnel()
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/http/client.py", line 929, in _tunnel
    raise OSError(f"Tunnel connection failed: {code} {message.strip()}")
OSError: Tunnel connection failed: 502 ERROR

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/requests/adapters.py", line 439, in send
    resp = conn.urlopen(
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/urllib3/

delete 1355285345797955586 2021-01-29 22:43:01
delete 1355136902261657605 2021-01-29 12:53:09
delete 1355077753670782976 2021-01-29 08:58:07
delete 1355136113023676420 2021-01-29 12:50:01
delete 1355346616757047296 2021-01-30 02:46:29
delete 1355346620464816134 2021-01-30 02:46:30
delete 1355338488099426304 2021-01-30 02:14:11
delete 1355378652209324035 2021-01-30 04:53:47
delete 1355337999765016579 2021-01-30 02:12:14
delete 1355444522138779652 2021-01-30 09:15:31
delete 1355442150385696776 2021-01-30 09:06:06
delete 1355475224590864385 2021-01-30 11:17:31
delete 1355447047437000705 2021-01-30 09:25:33
delete 1355462841474580486 2021-01-30 10:28:19
delete 1355502579879297029 2021-01-30 13:06:13
delete 1355503097657741317 2021-01-30 13:08:17
delete 1355503437291507712 2021-01-30 13:09:38
delete 1355499769343344643 2021-01-30 12:55:03
delete 1355505354939920385 2021-01-30 13:17:15
delete 1355504135819280391 2021-01-30 13:12:24
delete 1355532882081574921 2021-01-30 15:06:38
delete 135553

...

Traceback (most recent call last):
  File "/tmp/ipykernel_620300/3195798785.py", line 52, in delete_tweet
    api.destroy_status(tweet["id"])
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/tweepy/api.py", line 46, in wrapper
    return method(*args, **kwargs)
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/tweepy/api.py", line 966, in destroy_status
    return self.request(
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/tweepy/api.py", line 259, in request
    raise Forbidden(resp)
tweepy.errors.Forbidden: 403 Forbidden
63 - User has been suspended.


delete 1361832847846309889 2021-02-17 00:20:27
delete 1361851418924097538 2021-02-17 01:34:14
delete 1361861417113059329 2021-02-17 02:13:58
delete 1361851909175345154 2021-02-17 01:36:11
delete 1361508335753732102 2021-02-16 02:50:57
delete 1361516694053658628 2021-02-16 03:24:10
delete 1361895549549486080 2021-02-17 04:29:36
delete 1361921751639158786 2021-02-17 06:13:43
delete 1361933017065816065 2021-02-17 06:58:29
delete 1361956454261952512 2021-02-17 08:31:37
delete 1361958160051761157 2021-02-17 08:38:23
delete 1362219259485515776 2021-02-18 01:55:54
delete 1362227692762894337 2021-02-18 02:29:25
delete 1362228156376092672 2021-02-18 02:31:16
delete 1362231317778194434 2021-02-18 02:43:49
mark 1362593859742568452 2021-02-19 02:44:26 deleted
delete 1362279450365227008 2021-02-18 05:55:05
delete 1363284308660613120 2021-02-21 00:28:02
delete 1363286603032981504 2021-02-21 00:37:09
delete 1363671453611134977 2021-02-22 02:06:24
delete 1363689807000469504 2021-02-22 03:19:20
delete 

Traceback (most recent call last):
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/urllib3/connectionpool.py", line 696, in urlopen
    self._prepare_proxy(conn)
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/urllib3/connectionpool.py", line 964, in _prepare_proxy
    conn.connect()
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/urllib3/connection.py", line 371, in connect
    self._tunnel()
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/http/client.py", line 929, in _tunnel
    raise OSError(f"Tunnel connection failed: {code} {message.strip()}")
OSError: Tunnel connection failed: 502 ERROR

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/requests/adapters.py", line 439, in send
    resp = conn.urlopen(
  File "/home/laisky/.pyenv/versions/3.9.7/lib/python3.9/site-packages/urllib3/

deletedelete 1364423978136207360 2021-02-24 03:56:40
 1364416268153982984 2021-02-24 03:26:02
delete 1364448290461294595 2021-02-24 05:33:17
delete 1364417420979396609 2021-02-24 03:30:37
delete 1364494849894850560 2021-02-24 08:38:17
delete 1364475576048775168 2021-02-24 07:21:42
delete 1364574451291983876 2021-02-24 13:54:36
delete 1364546498038538240 2021-02-24 12:03:31
delete 1364574923004411907 2021-02-24 13:56:28
delete 1364745247868157952 2021-02-25 01:13:17
delete 1364749015649193985 2021-02-25 01:28:15
delete 1364785327127154691 2021-02-25 03:52:33
delete 1364754713187524609 2021-02-25 01:50:54
delete 1364546037604438019 2021-02-24 12:01:42
delete 1364780745303072768 2021-02-25 03:34:20
delete 1364798474936455168 2021-02-25 04:44:47
delete 1364575938759983109 2021-02-24 14:00:31
delete 1364806954082652161 2021-02-25 05:18:29
delete 1364799848352944133 2021-02-25 04:50:15
delete 1364800886279561216 2021-02-25 04:54:22
delete 1364856754278858754 2021-02-25 08:36:22
delete 136492

KeyboardInterrupt: 

In [ ]:
# download videos

import requests


requests.get('https://video.twimg.com/ext_tw_video/1180858517940396035/pu/vid/864x486/84gdcqfeV-ud1LgC.mp4?tag=10')

In [ ]:
# download image
from pathlib import Path

import requests

dirpath = r"/var/www/uploads/twitter"
# dirpath = r'/Users/laisky/Downloads'


def download_images_for_tweet(tweet):
    for img in tweet["entities"]["media"]:
        with requests.get(img["media_url_https"] + ":orig") as r:
            if r.status_code != 200:
                print(f"download error: [{r.status_code}]{r.content}")
                continue

            fpath = Path(dirpath, img["media_url_https"].split("/")[-1])
            if fpath.is_file():
                continue

            with open(fpath, "wb") as f:
                f.write(r.content)

            print("tweet img ok", tweet["id"], fpath)


def download_images():
    for tweet in tweets.find(
        {"entities.media": {"$exists": 1}}, no_cursor_timeout=True
    ).sort("_id", -1):
        download_images_for_tweet(tweet)


download_images()


In [ ]:
# download related tweets


def get_tweet_text(tweet: Dict[str, any]) -> str:
    return tweet.get("full_text") or tweet.get("text")


def twitter_api_parser(tweet: Dict[str, any]) -> Dict[str, any]:
    """Parse tweet document got from twitter api"""
    reg_topic = re.compile(r"[\b|\s]#(\S+)")
    tweet["topics"] = reg_topic.findall(get_tweet_text(tweet).replace(".", "_"))
    tweet["created_at"] = datetime.datetime.strptime(
        tweet["created_at"], "%a %b %d %H:%M:%S +0000 %Y"
    )

    # replace url
    t = get_tweet_text(tweet)
    if tweet.get("entities"):
        # parse entities media
        entities = tweet.get("extended_entities") or tweet.get("entities")
        if "media" in entities:
            for media in tweet["entities"]["media"]:
                for surl in ["url", "display_url"]:
                    durl = media.get("media_url_https") or media["media_url"]
                    t = t.replace(surl, durl)

        # parse entities urls
        if "urls" in tweet["entities"]:
            for d in tweet["entities"]["urls"]:
                for surl in ["url", "display_url"]:
                    eurl = d["expanded_url"]
                    t = t.replace(surl, eurl)

        tweet["text"] = t

    return tweet


def gen_related_tweets(
    tweetCol: pymongo.collection.Collection, tweet: Dict[str, any]
) -> Generator[str, None, None]:
    related_ids = []
    tweet.get("in_reply_to_status_id") and related_ids.append(
        tweet["in_reply_to_status_id"]
    )
    tweet.get("retweeted_status") and related_ids.append(
        tweet["retweeted_status"]["id"]
    )
    tweet.get("quoted_status") and related_ids.append(tweet["quoted_status"]["id"])
    for _id in filter(lambda id_: not tweetCol.find_one({"id": id_}), related_ids):
        yield _id


def save_relate_tweets(status):
    related_ids = []
    status.get("in_reply_to_status_id") and related_ids.append(
        status["in_reply_to_status_id"]
    )
    status.get("retweeted_status") and related_ids.append(
        status["retweeted_status"]["id"]
    )
    status.get("quoted_status") and related_ids.append(status["quoted_status"]["id"])
    related_ids = filter(lambda id_: not tweets.find_one({"id": id_}), related_ids)

    for id_ in related_ids:
        try:
            docu = api.get_status(id_, tweet_mode="extended")
        except Exception as err:
            print(f"load tweet {id_} got error: {err}")
        else:
            print(f"save tweet [{docu['user']['screen_name']}]{docu['id']}")
            save_tweet(docu)
            save_relate_tweets(docu)


def save_tweet(docu):
    docu = twitter_api_parser(docu)
    tweets.update_one({"id": docu["id"]}, {"$set": docu}, upsert=True)


def download_relate_tweets():
    for tweet in tweets.find(no_cursor_timeout=True).sort("_id", -1):
        save_relate_tweets(tweet)


# download_relate_tweets()


In [ ]:
# replace all urls


def replace_to_laisky_url(url):
    url = url.replace(
        "http://pbs.twimg.com/media/", "https://s3.laisky.com/uploads/twitter/"
    )
    url = url.replace(
        "https://pbs.twimg.com/media/", "https://s3.laisky.com/uploads/twitter/"
    )

    return url


def replace_media_urls(tweet):
    for ee in ["entities", "extended_entities"]:
        for media in tweet.get(ee, {}).get("media", []):
            surl = media.get("url", "")
            durl = media.get("media_url_https", "")
            durl = replace_to_laisky_url(durl)
            if durl != "":
                tweet["text"] = tweet["text"].replace(surl, durl)


def replace_short_urls(tweet):
    for media in tweet.get("entities", {}).get("urls", []):
        surl = media.get("url", "")
        durl = media.get("expanded_url", "")
        if durl != "":
            tweet["text"] = tweet["text"].replace(surl, durl)


def replace_all_urls():
    i = 0
    for tweet in tweets.find(no_cursor_timeout=True).sort("_id", 1):
        replace_media_urls(tweet)
        replace_short_urls(tweet)

        #         i += 1
        #         print(tweet['text'])
        #         if i > 10:
        #             return
        #         else:
        #             continue

        tweets.update_one({"id": tweet["id"]}, {"$set": tweet})


replace_all_urls()


In [ ]:
# save users

def save_users():
    for tweet in tweets.find(no_cursor_timeout=True).sort("_id", 1):
        if tweet.get("user") is None:
            continue

        user = tweet["user"]
        users.update_one({"id": user["id"]}, {"$set": user}, upsert=True)


save_users()

In [ ]:
# remove duplicate tweets

def remove_duplicate_tweets():
    for tweet in tweets.find(no_cursor_timeout=True).sort("_id", -1):
        tweets.delete_many({"id_str": tweet["id_str"], "_id": {"$ne": tweet["_id"]}})


def add_id_str():
    for tweet in tweets.find(no_cursor_timeout=True).sort("_id", -1):
        tweets.update_one(
            {"_id": tweet["_id"]},
            {"$set": {"id_str": str(tweet["id"]), "id": int(tweet["id"])}},
        )


def add_user_id_str():
    for docu in users.find(no_cursor_timeout=True).sort("_id", -1):
        users.update_one(
            {"_id": docu["_id"]},
            {"$set": {"id_str": str(docu["id"]), "id": int(docu["id"])}},
        )


add_id_str()
add_user_id_str()
remove_duplicate_tweets()

In [ ]:
s = 'https://twitter.com/aftergreatest/status/1382892663754551298'

